In [ ]:
!nvidia-smi


Sun Jan 18 13:23:13 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   31C    P0             44W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
!pip -q install -U transformers accelerate sentencepiece pandas tqdm vllm huggingface_hub


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.9/87.9 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 126.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 149.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.9/474.9 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.0/355.0 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.0/183.0 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
from huggingface_hub import login
login()


In [ ]:
!git clone https://github.com/chen700564/RGB
!ls RGB
!ls RGB/data


Cloning into 'RGB'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 50 (delta 22), reused 34 (delta 10), pack-reused 0 (from 0)
Receiving objects: 100% (50/50), 8.16 MiB | 11.59 MiB/s, done.
Resolving deltas: 100% (22/22), done.
config	env.sh	   fact_evalue.py  models     reject_evalue.py
data	evalue.py  LICENSE.txt	   readme.md
en_fact.json  en.json	      zh_fact.json  zh.json
en_int.json   en_refine.json  zh_int.json   zh_refine.json


In [ ]:
%%writefile rgb_eval.py
import argparse
import json
import os
import random
from dataclasses import dataclass
from typing import Any, Dict, List, Optional

import pandas as pd
from tqdm import tqdm

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer


# -----------------------------
# Prompt (Figure 3 from the paper)
# -----------------------------
SYSTEM_INSTRUCTION_EN = (
    "You are an accurate and reliable AI assistant that can\n"
    "answer questions with the help of external documents.\n"
    "Please note that external documents may contain noisy\n"
    "or factually incorrect information. If the information in\n"
    "the document contains the correct answer, you will give\n"
    "an accurate answer. If the information in the document\n"
    "does not contain the answer, you will generate 'I can not\n"
    "answer the question because of the insufficient\n"
    "information in documents.' If there are inconsistencies\n"
    "with the facts in some of the documents, please generate\n"
    "the response 'There are factual errors in the provided\n"
    "documents.' and provide the correct answer."
)

USER_TEMPLATE_EN = "Document:\n{DOCS}\n\nQuestion:\n{QUERY}"

REJECTION_PHRASE = "I can not answer the question because of the insufficient information in documents."
FACT_ERROR_PHRASE = "There are factual errors in the provided documents."


def format_docs(docs: List[str]) -> str:
    out = []
    for i, d in enumerate(docs, 1):
        out.append(f"[Doc {i}] {str(d).strip()}")
    return "\n\n".join(out)


def build_prompt(question: str, docs: List[str]) -> str:
    return USER_TEMPLATE_EN.format(DOCS=format_docs(docs), QUERY=question)


# -----------------------------
# Dataset parsing (robust to schema differences)
# -----------------------------
@dataclass
class RGBExample:
    qid: str
    question: str
    answers: List[str]
    pos_docs: List[str]
    neg_docs: List[str]
    need_reject: Optional[int]
    is_counterfactual: Optional[int]
    raw: Dict[str, Any]


def _as_list(x) -> List[Any]:
    if x is None:
        return []
    return x if isinstance(x, list) else [x]


def infer_and_parse_examples(path: str) -> List[RGBExample]:
    # Support JSONL (one JSON dict per line) and JSON array
    with open(path, "r", encoding="utf-8") as f:
        first = f.read(1)
        f.seek(0)
        if first == "[":
            data = json.load(f)
        else:
            data = []
            for line in f:
                line = line.strip()
                if not line:
                    continue
                data.append(json.loads(line))

    if not isinstance(data, list):
        raise ValueError("RGB file must decode to a list of examples")

    q_keys = ["question", "query", "Q", "Question"]
    a_keys = ["ground_truths", "answers", "answer", "ground_truth", "A"]
    pos_keys = ["positive", "pos", "positive_docs", "pos_docs"]
    neg_keys = ["negative", "neg", "negative_docs", "neg_docs"]

    reject_keys = ["need_reject", "should_reject", "is_reject", "reject"]
    cf_keys = ["is_counterfactual", "counterfactual", "is_cf"]

    def pick(ex: dict, keys: List[str]) -> Optional[Any]:
        for k in keys:
            if k in ex:
                return ex[k]
        return None

    def pick_bool(ex: dict, keys: List[str]) -> Optional[int]:
        for k in keys:
            if k in ex:
                v = ex[k]
                if isinstance(v, bool):
                    return int(v)
                if isinstance(v, (int, float)):
                    return int(v != 0)
                if isinstance(v, str):
                    s = v.strip().lower()
                    if s in ["true", "yes", "1"]:
                        return 1
                    if s in ["false", "no", "0"]:
                        return 0
        return None

    def normalize_docs(docs_any) -> List[str]:
        docs_any = _as_list(docs_any)
        out_docs: List[str] = []
        for d in docs_any:
            if isinstance(d, str):
                out_docs.append(d)
            elif isinstance(d, dict):
                for dk in ["content", "text", "snippet", "document", "doc"]:
                    if dk in d and isinstance(d[dk], str):
                        out_docs.append(d[dk])
                        break
        return out_docs

    out_examples: List[RGBExample] = []
    for idx, ex in enumerate(data):
        if not isinstance(ex, dict):
            continue

        q = pick(ex, q_keys)
        a = pick(ex, a_keys)
        pos = pick(ex, pos_keys)
        neg = pick(ex, neg_keys)

        question = q if isinstance(q, str) else ""

        # Flatten answers
        answers: List[str] = []
        for item in _as_list(a):
            if isinstance(item, str):
                answers.append(item)
            elif isinstance(item, list):
                answers.extend([x for x in item if isinstance(x, str)])

        pos_docs = normalize_docs(pos)
        neg_docs = normalize_docs(neg)

        # Fallback if docs stored unlabelled
        if not pos_docs and not neg_docs:
            for k in ["documents", "docs", "contexts", "context"]:
                if k in ex:
                    neg_docs = normalize_docs(ex[k])
                    break

        need_reject = pick_bool(ex, reject_keys)
        is_counterfactual = pick_bool(ex, cf_keys)

        qid = str(ex.get("id", ex.get("qid", idx)))
        out_examples.append(
            RGBExample(
                qid=qid,
                question=question,
                answers=answers,
                pos_docs=pos_docs,
                neg_docs=neg_docs,
                need_reject=need_reject,
                is_counterfactual=is_counterfactual,
                raw=ex,
            )
        )

    return out_examples


# -----------------------------
# Sampling docs for noise experiments
# -----------------------------
def sample_docs(ex: RGBExample, passage_num: int, noise_rate: float, rng: random.Random) -> List[str]:
    n_noise = int(round(passage_num * noise_rate))
    n_pos = passage_num - n_noise

    def pick(pool: List[str], k: int) -> List[str]:
        if k <= 0:
            return []
        if not pool:
            return [""] * k
        if len(pool) >= k:
            return rng.sample(pool, k)
        return [pool[rng.randrange(len(pool))] for _ in range(k)]

    docs = pick(ex.pos_docs, n_pos) + pick(ex.neg_docs, n_noise)
    rng.shuffle(docs)
    return docs


# -----------------------------
# Model runner
# -----------------------------
class HFGenerator:
    def __init__(self, model_name: str):
        self.model_name = model_name
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
            device_map="auto" if torch.cuda.is_available() else None,
        )
        self.model.eval()

    def generate(
        self,
        system_text: str,
        user_text: str,
        max_new_tokens: int,
        temperature: float,
        top_p: float,
        seed: int,
    ) -> str:
        torch.manual_seed(seed)

        messages = [
            {"role": "system", "content": system_text},
            {"role": "user", "content": user_text},
        ]

        if hasattr(self.tokenizer, "apply_chat_template"):
            enc = self.tokenizer.apply_chat_template(
                messages,
                add_generation_prompt=True,
                return_tensors="pt",
                return_dict=True,
            )
            input_ids = enc["input_ids"].to(self.model.device)
            attention_mask = enc.get("attention_mask", None)
            if attention_mask is None:
                attention_mask = torch.ones_like(input_ids, device=self.model.device)
            else:
                attention_mask = attention_mask.to(self.model.device)
        else:
            joined = f"{system_text}\n\n{user_text}\n\nAnswer:"
            enc = self.tokenizer(joined, return_tensors="pt")
            input_ids = enc["input_ids"].to(self.model.device)
            attention_mask = enc["attention_mask"].to(self.model.device)

        with torch.no_grad():
            out = self.model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_new_tokens=max_new_tokens,
                do_sample=(temperature > 0),
                temperature=temperature if temperature > 0 else None,
                top_p=top_p,
                pad_token_id=self.tokenizer.pad_token_id,
                eos_token_id=self.tokenizer.eos_token_id,
            )

        gen_ids = out[0][input_ids.shape[-1] :]
        text = self.tokenizer.decode(gen_ids, skip_special_tokens=True).strip()
        return text


# -----------------------------
# Metrics
# -----------------------------
def contains_any_answer(gen: str, answers: List[str]) -> bool:
    g = gen.lower()
    for a in answers:
        if a and a.lower() in g:
            return True
    return False


def is_rejection(gen: str) -> bool:
    return REJECTION_PHRASE.lower() in gen.lower()


def is_error_detected(gen: str) -> bool:
    return FACT_ERROR_PHRASE.lower() in gen.lower()


def summarize(df: pd.DataFrame, task: str) -> Dict[str, float]:
    if len(df) == 0:
        return {}

    if task in ["noise", "integration"]:
        return {"accuracy": float(df["correct"].mean())}

    if task == "rejection":
        # Paper-style: evaluate only where the dataset says we SHOULD reject
        if "need_reject" in df.columns and df["need_reject"].notna().any():
            mask = df["need_reject"] == 1
            rej = float(df.loc[mask, "is_rejection"].mean()) if mask.any() else float("nan")
            false_rej = float(df.loc[~mask, "is_rejection"].mean()) if (~mask).any() else float("nan")
            return {"rejection_rate": rej, "false_rejection_rate": false_rej}
        return {"rejection_rate": float(df["is_rejection"].mean())}

    if task == "counterfactual":
        out: Dict[str, float] = {}

        # ACC: no-doc correctness
        if "correct_nodoc" in df.columns and df["correct_nodoc"].notna().any():
            out["accuracy_nodoc"] = float(df["correct_nodoc"].mean())

        # ACCdoc: with-doc correctness
        out["accuracy_doc"] = float(df["correct"].mean())

        # ED/CR conditioned on counterfactual examples
        if "is_counterfactual" in df.columns and df["is_counterfactual"].notna().any():
            cf_mask = df["is_counterfactual"] == 1
        else:
            cf_mask = pd.Series([True] * len(df), index=df.index)

        if cf_mask.any():
            ed = float(df.loc[cf_mask, "is_error_detected"].mean())
            cr = float(((df.loc[cf_mask, "is_error_detected"] == 1) & (df.loc[cf_mask, "correct"] == 1)).mean())
        else:
            ed, cr = float("nan"), float("nan")

        out["error_detection_rate"] = ed
        out["error_correction_rate"] = cr
        return out

    return {}


def run_split(
    task: str,
    split_path: str,
    gen: HFGenerator,
    out_dir: str,
    passage_num: int,
    noise_rate: float,
    temperature: float,
    top_p: float,
    max_new_tokens: int,
    seed: int,
) -> pd.DataFrame:
    os.makedirs(out_dir, exist_ok=True)
    rng = random.Random(seed)
    examples = infer_and_parse_examples(split_path)

    rows: List[Dict[str, Any]] = []
    out_jsonl = os.path.join(out_dir, "generations.jsonl")

    with open(out_jsonl, "w", encoding="utf-8") as f:
        for ex in tqdm(examples, desc=f"{task}:{os.path.basename(split_path)}:{noise_rate}"):
            # For counterfactual: compute ACC (no-doc) by an extra generation pass
            gen_nodoc = None
            correct_nodoc = None
            if task == "counterfactual":
                nodoc_prompt = build_prompt(ex.question, [])
                gen_nodoc = gen.generate(
                    SYSTEM_INSTRUCTION_EN,
                    nodoc_prompt,
                    max_new_tokens=max_new_tokens,
                    temperature=temperature,
                    top_p=top_p,
                    seed=seed,
                )
                correct_nodoc = int(contains_any_answer(gen_nodoc, ex.answers))

            docs = sample_docs(ex, passage_num=passage_num, noise_rate=noise_rate, rng=rng)
            prompt = build_prompt(ex.question, docs)

            text = gen.generate(
                SYSTEM_INSTRUCTION_EN,
                prompt,
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                top_p=top_p,
                seed=seed,
            )

            correct = int(contains_any_answer(text, ex.answers))
            rec = {
                "id": ex.qid,
                "question": ex.question,
                "answers": ex.answers,
                "noise_rate": noise_rate,
                "need_reject": ex.need_reject,
                "is_counterfactual": ex.is_counterfactual,
                "generation": text,
                "correct": correct,
                "is_rejection": int(is_rejection(text)),
                "is_error_detected": int(is_error_detected(text)),
            }

            if gen_nodoc is not None:
                rec["generation_nodoc"] = gen_nodoc
                rec["correct_nodoc"] = correct_nodoc

            rows.append(rec)
            f.write(json.dumps(rec, ensure_ascii=False) + "\n")

    df = pd.DataFrame(rows)
    df.to_csv(os.path.join(out_dir, "per_example.csv"), index=False)
    return df


def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--data_dir", required=True)
    ap.add_argument("--out_dir", required=True)
    ap.add_argument("--model", required=True)

    ap.add_argument("--passage_num", type=int, default=5)
    ap.add_argument("--temperature", type=float, default=0.2)
    ap.add_argument("--top_p", type=float, default=0.95)
    ap.add_argument("--max_new_tokens", type=int, default=256)
    ap.add_argument("--seed", type=int, default=42)

    args = ap.parse_args()

    suite = [
        ("noise", "en_refine.json", [0.0, 0.2, 0.4, 0.6, 0.8]),
        ("rejection", "en_refine.json", [1.0]),
        ("integration", "en_int.json", [0.0]),
        ("counterfactual", "en_fact.json", [0.0]),
    ]

    gen = HFGenerator(args.model)

    summary_rows: List[Dict[str, Any]] = []
    for task, fname, noise_rates in suite:
        split_path = os.path.join(args.data_dir, fname)
        if not os.path.exists(split_path):
            print(f"[WARN] missing {split_path}, skipping.")
            continue

        for nr in noise_rates:
            run_dir = os.path.join(
                args.out_dir,
                args.model.replace("/", "__"),
                f"{task}_{fname.replace('.json', '')}",
                f"noise_{nr}",
            )

            df = run_split(
                task=task,
                split_path=split_path,
                gen=gen,
                out_dir=run_dir,
                passage_num=args.passage_num,
                noise_rate=nr,
                temperature=args.temperature,
                top_p=args.top_p,
                max_new_tokens=args.max_new_tokens,
                seed=args.seed,
            )
            met = summarize(df, task)
            summary_rows.append(
                {
                    "model": args.model,
                    "task": task,
                    "file": fname,
                    "noise_rate": nr,
                    "n": len(df),
                    **met,
                }
            )

    summary_df = pd.DataFrame(summary_rows)
    os.makedirs(args.out_dir, exist_ok=True)
    summary_df.to_csv(os.path.join(args.out_dir, "summary.csv"), index=False)
    print(summary_df)


if __name__ == "__main__":
    main()


Writing rgb_eval.py


In [ ]:
!find /content -name rgb_eval.py


/content/rgb_eval.py


In [ ]:
!python /content/rgb_eval.py --data_dir RGB/data --out_dir /content/runs_rgb --model Qwen/Qwen2.5-7B-Instruct

tokenizer_config.json: 7.30kB [00:00, 3.87MB/s]
vocab.json: 2.78MB [00:00, 35.9MB/s]
merges.txt: 1.67MB [00:00, 58.2MB/s]
tokenizer.json: 7.03MB [00:00, 29.4MB/s]
config.json: 100% 663/663 [00:00<00:00, 6.19MB/s]
`torch_dtype` is deprecated! Use `dtype` instead!
2026-01-18 13:25:40.840920: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-18 13:25:40.860720: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768742740.880049    4318 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768742740.885422    4318 cuda_blas.cc:1407] U

In [ ]:
!python rgb_eval.py --data_dir RGB/data --out_dir "./runs_rgb" --model mistralai/Mistral-7B-Instruct-v0.3

tokenizer_config.json: 141kB [00:00, 205MB/s]
tokenizer.model: 100% 587k/587k [00:01<00:00, 450kB/s]
tokenizer.json: 1.96MB [00:00, 199MB/s]
special_tokens_map.json: 100% 414/414 [00:00<00:00, 385kB/s]
config.json: 100% 601/601 [00:00<00:00, 4.36MB/s]
`torch_dtype` is deprecated! Use `dtype` instead!
2026-01-18 14:20:10.480344: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-18 14:20:10.496508: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768746010.516242   18260 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768746

In [ ]:
!python rgb_eval.py --data_dir RGB/data --out_dir "./runs_rgb" --model microsoft/Phi-3.5-mini-instruct --max_new_tokens 128 --temperature 0.2

tokenizer_config.json: 3.98kB [00:00, 16.3MB/s]
tokenizer.model: 100% 500k/500k [00:01<00:00, 416kB/s]
tokenizer.json: 1.84MB [00:00, 51.3MB/s]
added_tokens.json: 100% 306/306 [00:00<00:00, 2.73MB/s]
special_tokens_map.json: 100% 665/665 [00:00<00:00, 4.45MB/s]
config.json: 3.45kB [00:00, 17.8MB/s]
`torch_dtype` is deprecated! Use `dtype` instead!
2026-01-18 15:31:11.500489: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-18 15:31:11.518817: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768750271.540442   36402 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one

In [ ]:
from huggingface_hub import login, HfApi
import os

login()  # paste token when prompted

repo_id = "vbupadhyaya/rgb-rag-run2"   # <-- change this
local_folder = "/content/runs_rgb"

api = HfApi()
# Create repo if it doesn't exist (type="dataset" recommended for results)
api.create_repo(repo_id=repo_id, repo_type="dataset", exist_ok=True)

# Upload entire folder
api.upload_folder(
    repo_id=repo_id,
    repo_type="dataset",
    folder_path=local_folder,
    path_in_repo="runs_rgb"
)

print("Uploaded to HF:", repo_id)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploaded to HF: vbupadhyaya/rgb-rag-run2


In [ ]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML

# ---- Update this path to wherever your runs are saved ----
SUMMARY_CSV = "/content/runs_rgb/summary.csv"

summary = pd.read_csv(SUMMARY_CSV)

# Keep a consistent model order
models = sorted(summary["model"].dropna().unique().tolist())

def fmt(df):
    """Format numbers as 2 decimals; show missing as --"""
    return df.applymap(lambda x: "--" if pd.isna(x) else f"{x:.2f}")

# ----------------------------
# 1) Noise Robustness (Accuracy %, English)
# ----------------------------
noise_rates_noise = [0.0, 0.2, 0.4, 0.6, 0.8]
noise_tbl = (
    summary[summary["task"].eq("noise")]
    .assign(acc_pct=lambda x: x["accuracy"] * 100)
    .pivot_table(index="model", columns="noise_rate", values="acc_pct", aggfunc="mean")
    .reindex(index=models, columns=noise_rates_noise)
)
noise_tbl.index.name = "Model"
noise_tbl.columns = [f"{c:.1f}" for c in noise_rates_noise]

# ----------------------------
# 2) Negative Rejection (Rejection Rate %, English)
#   Paper wants Rej only (no Rej*)
#   In our patched eval, "rejection_rate" is conditioned on need_reject==1
# ----------------------------
rej_tbl = (
    summary[summary["task"].eq("rejection")]
    .groupby("model")["rejection_rate"]
    .mean()
    .mul(100)
    .reindex(models)
    .to_frame("Rej")
)
rej_tbl.index.name = "Model"

# ----------------------------
# 3) Information Integration (Accuracy %, English)
#   Paper shows 0.0, 0.2, 0.4 columns, but HF en_int often only has 0.0
# ----------------------------
noise_rates_integ = [0.0, 0.2, 0.4]
integ_tbl = (
    summary[summary["task"].eq("integration")]
    .assign(acc_pct=lambda x: x["accuracy"] * 100)
    .pivot_table(index="model", columns="noise_rate", values="acc_pct", aggfunc="mean")
    .reindex(index=models, columns=noise_rates_integ)
)
integ_tbl.index.name = "Model"
integ_tbl.columns = [f"{c:.1f}" for c in noise_rates_integ]

# ----------------------------
# 4) Counterfactual Robustness (%, English)
#   Paper columns: ACC, ACCdoc, ED, CR
#   Our summary includes:
#     accuracy_nodoc -> ACC
#     accuracy_doc   -> ACCdoc
#     error_detection_rate -> ED
#     error_correction_rate -> CR
# ----------------------------
cf = summary[summary["task"].eq("counterfactual")].copy()

cf_tbl = pd.DataFrame(index=models, columns=["ACC", "ACCdoc", "ED", "CR"], dtype=float)
cf_tbl.index.name = "Model"

if "accuracy_nodoc" in cf.columns:
    cf_tbl["ACC"] = cf.groupby("model")["accuracy_nodoc"].mean().reindex(models) * 100
if "accuracy_doc" in cf.columns:
    cf_tbl["ACCdoc"] = cf.groupby("model")["accuracy_doc"].mean().reindex(models) * 100

if "error_detection_rate" in cf.columns:
    cf_tbl["ED"] = cf.groupby("model")["error_detection_rate"].mean().reindex(models) * 100
if "error_correction_rate" in cf.columns:
    cf_tbl["CR"] = cf.groupby("model")["error_correction_rate"].mean().reindex(models) * 100

# ----------------------------
# Print EXACT paper-style tables
# ----------------------------
print("\nNoise Robustness (Accuracy %, English)\n")
display(fmt(noise_tbl))

print("\nNegative Rejection (Rejection Rate %, English)\n")
display(fmt(rej_tbl))

print("\nInformation Integration (Accuracy %, English)\n")
display(fmt(integ_tbl))

print("\nCounterfactual Robustness (%, English)\n")
display(fmt(cf_tbl))

# ----------------------------
# Side-by-side layout (like the PDF)
# ----------------------------
def card(df, title):
    return f"""
    <div style="flex:1; min-width:360px; padding:12px; border:1px solid #ddd; border-radius:10px;">
        <div style="font-weight:700; margin-bottom:8px;">{title}</div>
        {fmt(df).to_html(escape=False)}
    </div>
    """

display(HTML(f"""
<div style="display:flex; gap:14px; flex-wrap:wrap;">
    {card(noise_tbl, "Noise Robustness (Accuracy %, English)")}
    {card(rej_tbl, "Negative Rejection (Rejection Rate %, English)")}
</div>
<div style="display:flex; gap:14px; flex-wrap:wrap; margin-top:14px;">
    {card(integ_tbl, "Information Integration (Accuracy %, English)")}
    {card(cf_tbl, "Counterfactual Robustness (%, English)")}
</div>
"""))
